# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wailua homesteads,22.0669,-159.3780,26.74,87,75,5.14,US,1730857770
1,1,rundu,-17.9333,19.7667,21.52,63,100,3.13,NaN,1730857826
2,2,ketchikan,55.3422,-131.6461,7.99,87,100,8.23,US,1730857828
3,3,kapuskasing,49.4169,-82.4331,5.05,93,100,3.60,CA,1730857738
4,4,pikalevo,59.5183,34.1664,-0.65,95,100,1.81,RU,1730857830


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points( 
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600, 
    scale = 1.25,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[((city_data_df["Max Temp"] <= 28.0) & (city_data_df["Max Temp"] >= 20.0))\
                                 & (city_data_df["Cloudiness"] == 0),:]

# Drop any rows with null values
ideal_df = ideal_df.dropna(how="any")

# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,aioun,16.6614,-9.6149,26.79,15,0,4.63,MR,1730857832
24,24,jaisalmer,26.9147,70.9181,25.96,37,0,3.12,IN,1730857853
51,51,koton-karfe,8.1000,6.8000,23.97,58,0,0.51,NG,1730857884
59,59,jamestown,42.0970,-79.2353,20.80,60,0,5.14,US,1730857673
80,80,tarfaya,27.9358,-12.9218,20.87,85,0,3.52,MA,1730857917


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

C:\Users\End User\AppData\Local\Temp\ipykernel_501148\3242012003.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
6,aioun,MR,16.6614,-9.6149,15,
24,jaisalmer,IN,26.9147,70.9181,37,
51,koton-karfe,NG,8.1000,6.8000,58,
59,jamestown,US,42.0970,-79.2353,60,
80,tarfaya,MA,27.9358,-12.9218,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "limit":10,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Searching hotels...")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Searching hotels...
aioun - nearest hotel: فندق العيون
jaisalmer - nearest hotel: Jeet Mahal
koton-karfe - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
tarfaya - nearest hotel: Hotel Tarfaya
sur - nearest hotel: Sur Hotel
fort bragg - nearest hotel: Airborne Inn Lodging
monte cristo - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
karor - nearest hotel: No hotel found
fada - nearest hotel: No hotel found
fortaleza - nearest hotel: Hotel Catedral
muridke - nearest hotel: Haji Hotel
nguigmi - nearest hotel: Guest PAM
tchintabaraden - nearest hotel: No hotel found
tengrela - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
abqaiq - nearest hotel: No hotel found
velingara - nearest hotel: Campement touristique
cabo san lucas - nearest hotel: Comfort Rooms
turabah - nearest hotel: No hotel found
hasilpur - nearest hotel: No hotel found
pauini - nearest hotel: No hotel found
jesup - nearest hotel: Days Inn
f

,City,Country,Lat,Lng,Humidity,Hotel Name
6,aioun,MR,16.6614,-9.6149,15,فندق العيون
24,jaisalmer,IN,26.9147,70.9181,37,Jeet Mahal
51,koton-karfe,NG,8.1000,6.8000,58,No hotel found
59,jamestown,US,42.0970,-79.2353,60,DoubleTree Jamestown
80,tarfaya,MA,27.9358,-12.9218,85,Hotel Tarfaya
92,sur,OM,22.5667,59.5289,79,Sur Hotel
93,fort bragg,US,35.1390,-79.0060,79,Airborne Inn Lodging
96,monte cristo,AR,-31.3431,-63.9444,40,No hotel found
98,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
126,karor,PK,31.2274,70.9492,30,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=650,
    frame_height=500,
    size="Humidity",
    color="City",
    hover_cols=["Hotel Name","Country"]
    )

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)